###### Текст распространяется на условиях лицензии Creative Commons Attribution license CC-BY 4.0, код — на условиях лицензии MIT license.  (c)2015 L.A. Barba, Pi-Yueh Chuang.

# Двумерный многозвенный профиль

Перед вами задание к четвертому модулю **Вихревой панельный метод для несущих тел** курса [_AeroPython_](https://github.com/barbagroup/AeroPython). Вы рассмотрите двумерный многозвенный профиль, или сечение механизированного крыла с выпущенным закрылком, применив панельный метод вихрей и источников из [занятия 11](11_Lesson11_vortexSourcePanelMethod.ipynb) к расчету обтекания пары профилей.

Главные особенности использования панельного метода для расчета многозвенных профилей состоят в следующем:

1. у каждому профилю соответствет вихрь с собственной интенсивностью, и
2. на задней кормке каждого профиля должно выполняться условие Кутты-Жуковского.

В первой части задания мы воспользуемся тестом, для которого существует теоретическое решение, полученное Вильямсом (1973). Посмотрите работу, на которую дана ссылка, и насладитесь полетом математической мысли! Во второй части вы будете иметь дело с более популярным профилем, NACA 23012, который будет использоваться в качестве основного и дополнительного звеньев. Для такой конфигураци есть экспериментальные данные, описанные в отчете Вензингера (1938). Это задание заставит вас задуматься о некоторых интересных, с точки зрения аэродинамики, вопросах.

## Часть 1: Тест Вильямса 

Ваша задача заключается в вычислении коэффициента давления на поверхностях многоэлементного профиля, состоящего из *основного профиля* и *внешнего закрылка*. Этот тест из работы Вильямса (1973), котороый получил красивое теоретическое решение при помощи конформных преобразований.

Профиль с выпущенным закрылком выглядит так:
<center> <img src="resources/MultiElementAirfoil.png" width="500"> </center>

Файлы CSV с $x$ и $y$ координатами узлов каждой панели содержатся в директории `resources` репозитория GitHub. Файлы имеют следующий формат наименования, в котором `***` представляет количество панелей `N`. `MainFoil_N=***.csv`, `FlapFoil_N=***.csv`.

Для отладки и тестирования можно использовать разбиение на меньшее число панелей, а для более серьезных вычислений — для ответа на вопросы задания — взять более подробное разбиение.

Также имеются два файла с именами `Cp_Main_theoretical.csv` и `Cp_Flap_theoretical.csv`, в которых содержатся теоретические значения — распределение коэффициента давления по оси $x$ для рассматриваемой конфигурации (закрылок отклонен на $30$º, основной профиль под нулевым углом атаки).

Теоретические значения коэффициентов подъемной силы и сопротивления для этого случая: $3.7386$ и $0$ соответственно (обезразмерены на скоростной напор).

### Подсказки

Вспомните [упражнение к занятию 11](11_Lesson11_Exercise.ipynb), в котором вас просили вывести математическую постановку панельного метода.

Теперь нужно повторить эти выкладки для двухэлементного профиля, начав с выражения для потенциала:

$$
\begin{split}
\phi(x, y) &= U_{\infty}x\cos\alpha + U_{\infty}y\sin\alpha \\
& + \int_{main} \frac{1}{2\pi} \sigma(s) \ln \sqrt{(x-\xi(s))^2+(y-\eta(s))^2} ds  \\
& + \int_{flap} \frac{1}{2\pi} \sigma(s) \ln \sqrt{(x-\xi(s))^2+(y-\eta(s))^2} ds \\
& - \int_{main} \frac{1}{2\pi} \gamma(s) \tan^{-1} \frac{y-\eta(s)}{x-\xi(s)} ds  \\
& - \int_{flap} \frac{1}{2\pi} \gamma(s) \tan^{-1} \frac{y-\eta(s)}{x-\xi(s)} ds
\end{split}
$$

Предположим следующее:

1. $\sigma(s)$ постоянны на каждой панели,
2. $\gamma(s)$ постоянны на каждом профиле, и
3. к основному профилю относятся панели с 1<sup>ой</sup> по $N_a$<sup>ю</sup>, а к закрылку — панели с $(N_a+1)$<sup>й</sup> по $N$<sup>ю</sup>, при этом $N=N_a+N_b$, а $N_b$ — это число панелей на закрылке.

Итого получится $N+2$ неизвестных: $\sigma_1\cdots\sigma_N$, $\gamma_a$, и $\gamma_b$. Величины  $\gamma_a$ и $\gamma_b$ соответствуют интенсивностям вихрей основного профиля и закрылка.

У вас должно получиться вывести матрицы для нормальной и касательной компонент скорости на $i$<sup>ой</sup> панели:

$$
U^n_i = b^n_i +
\left[\begin{smallmatrix}A^n_{i1}\cdots A^n_{iN}\end{smallmatrix}, \sum_{j=1}^{N_a}B^n_{ij}, \sum_{j=N_a+1}^{N}B^n_{ij}\right]
\left[\begin{smallmatrix}\sigma_1 \\ \vdots \\ \sigma_N \\ \gamma_a \\ \gamma_b\end{smallmatrix}\right]
$$

$$
U^t_i = b^t_i +
\left[\begin{smallmatrix}A^t_{i1}\cdots A^t_{iN}\end{smallmatrix}, \sum_{j=1}^{N_a}B^t_{ij}, \sum_{j=N_a+1}^{N}B^t_{ij}\right]
\left[\begin{smallmatrix}\sigma_1 \\ \vdots \\ \sigma_N \\ \gamma_a \\ \gamma_b\end{smallmatrix}\right]
$$

Используя граничное условие непротекания на панелях с $1$<sup>ой</sup> по $N$<sup>ую</sup>, вы получите $N$ линейных уравнений. Для нахождения всех $N+2$ неизвестных нужно еще два условия Кутты-Жуковского для двух профилей:

$$
U^t_{1} = U^t_{N_a} \\
U^t_{N_a+1} = U^t_{N}
$$

И теперь вы готовы к расчету потенциального течения вокруг двумерного многозвенного крыла!

##### Вопросы

Запустите ваш код, используя 100 панелей на основном профиле и закрылке и ответьте на следующие вопросы.

### Вопрос 1:

* Чему равен коэффициент давления на $3$<sup>й</sup> панели *основного профиля*?

### Вопрос 2:

* Чему равен коэффициент давления на $10$<sup>й</sup> панели *закрылка*?*?

### Вопрос 3:

* Чему равна подъемная сила?

**Подсказка:** $~L = - \oint_{main}p\vec{n}\cdot\vec{j}dl- \oint_{flap}p\vec{n}\cdot\vec{j}dl$

### Вопрос 4:

* Какова величина сопротивления?

### Вопрос 5:

* Запустите программу с 200 панелями на основном профиле и закрылке. На сколько процентов уменьшилась ошибка в определении подъемной силы?

## Часть 2: Тест Вензингера

Теперь у вас должен появиться панельный солвер для двухкомпонентного профиля, который можно использовать повторно. Следующий шаг — применить его к более реалистичной задаче. Нам доступны экспериментальные данные (Wenzinger, 1938) для профиля NACA 23012 с закрылком той же формы, с которыми можно сравнить результаты расчетов.

На следующем рисунке показан профиль сечения крыла с 0º отклонением закрылка:

<center> <img src="resources/MultiElementAirfoil.png" width="500"> </center>

Файлы CSV с $x$ и $y$ координатами узлов каждой панели этого сечения крыла со 150 панелями на каждом профиле: `NACA23012_MainFoil.csv` и `NACA23012_FlapFoil.csv` в директории `resources`.

Точка вращения закрылка расположена в точке $(1.03, -0.054)$ (см. конфигурацию на рис. 1). Используя координаты точки вращения, можно получить различные конфигурации с разными углами отклонения закрылков.

Попробуйте использовать разные углы отклонения закрылков при различных углах атаки (по хордн основного профиля) и сравните с экспериментальными результатами Вензингера. Напомним, как определяется полный коэффициент подъемной силы $L/(l_{main}+l_{flap})$, где $L$  — подъемная сила (на единицу размаха), как и в рассмотренной выше задаче, а $l_{main}$ и $l_{flap}$ — хорды обоих профилей. **Подумайте, что может быть источником расхождения между вашими результатами и экспериментальными данными**.

##### Вопросы

### Вопрос 1:

* Используйте [метод деления отрезка пополам](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%B1%D0%B8%D1%81%D0%B5%D0%BA%D1%86%D0%B8%D0%B8) для определения угла атаки $\alpha$, при котором подъемная сила равна нулю при отклонении закрылка $20$º. Округлите ответ до второго знака после плавающей точки.

**Подсказка**: можно начать поиск в интервале $-14\le\alpha\le14$.

### Вопрос 2:

* Используйте [метод деления отрезка пополам](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%B1%D0%B8%D1%81%D0%B5%D0%BA%D1%86%D0%B8%D0%B8) для определения угла отклонения закрылка, при котором подъемная сила равна нулю при угле атаки $\alpha = 0$º. Округлите ответ до второго знака после плавающей точки.

### Вопрос 3:

* Предположим, что основной профиль крепится к фюзеляжу самолета под угорм атаки $4$º (то есть строительный угол атаки на котором проходит крейсерский полет, равен $4$º). Вычислите коэффициенты подъемной силы с закрылком, отклоненным на $0$º,  а также на $5$, $10$ и $15$ градусов. На сколько медленнее (в процентах) будет лететь самолет с выпущенными закрылками, по сравнению со случаем, когда они убраны?

##### Подумайте

Каков физический смысл результата, полученного в **Вопросе 3**? Зачем нужен закрылок?

## Ссылки

* B. R. Williams (1973), *An Exact Test Case for the Plane Potential Flow About Two Adjacent Lifting Aerofoils*, Reports & Memoranda No. 3717, Aeronautical Research Council of the United Kingdom // [PDF на сайте университета Крэнфилда](http://naca.central.cranfield.ac.uk/reports/arc/rm/3717.pdf)

* C. J. Wenzinger (1938), *Pressure distribution over an NACA 23012 airfoil with an N.A.C.A. 23012 external-airfoil flap*, NACA Technical Report No.614 // [PDF на сайте NASA](http://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19930091690.pdf)

-----------------------------

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()